In [1]:
from dataclasses import dataclass
from vi import Agent, Config, Simulation, Vector2

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
@dataclass
class FlockingConfig(Config):
    alignment_weight: float = 1.1
    cohesion_weight: float = 0.03
    separation_weight: float = 3.1

class FlockingAgent(Agent):
    def on_spawn(self):
        self.collision_cooldown_threshhold = 20 # number of ticks that agent will not change direction after a collision
        self.collision_cooldown = 0 # number of ticks left before agent can change direction after a collision

    def on_obstacle_collision(self):
        
        self.move = -self.move  # 180 turn
        self.pos += self.move * self.config.movement_speed
        self.collision_cooldown = self.collision_cooldown_threshhold 
        self.collision_reflection_retry = 0

    def change_position(self):
        self.there_is_no_escape()
        if self.collision_cooldown > 0:
            self.pos += self.move
            self.collision_cooldown -= 1
            return 
        
        intersections = list(self.obstacle_intersections(scale = 1.0))

        if intersections:
            self.on_obstacle_collision()
            return

        neighbors = list(self.in_proximity_accuracy())

        if not neighbors:
            self.pos += self.move 
            return
        
        separationVelocity = Vector2() # move away from nearby agents
        alignmentVelocity = Vector2() # match the average direction of nearby agents
        cohesionVelocity = Vector2() # move toward the average position of nearby agents
        num_neighbors = len(neighbors)

        for neighbor, distance in neighbors: # loop trough each nearby agent.
            offset = self.pos - neighbor.pos # a vector pointing away from the neighbor
            if distance > 0:
                separationVelocity += offset / (distance**1.3) # push neighbors away

            alignmentVelocity += neighbor.move # sum al neighbors movement direction
            cohesionVelocity += neighbor.pos # sum all neigbors position

        # take the average of each vector to get a single direction fro each behavior
        separationVelocity /= num_neighbors
        alignmentVelocity /= num_neighbors
        cohesionVelocity = (cohesionVelocity / num_neighbors - self.pos)

        # multiply bt weights
        separationVelocity *= self.config.separation_weight
        alignmentVelocity *= self.config.alignment_weight
        cohesionVelocity *= self.config.cohesion_weight

        # single movement vector for all behaviors
        self.move = (separationVelocity + alignmentVelocity + cohesionVelocity).normalize() * self.config.movement_speed
        self.pos += self.move

config = FlockingConfig(image_rotation = True, movement_speed = 2.0, radius = 50, seed = 777, duration = 10000, fps_limit = 60)
x, y = config.window.as_tuple()

(
    Simulation(config)
    .spawn_obstacle("images/triangle@200px.png", x // 2, y // 2)
    .batch_spawn_agents(100, FlockingAgent, images=["images/triangle.png"])
    .run()
)

: 